# Préambule

## Introduction & Contexte

Ce projet a été réalisé par 3 étudiants de Master 1 à l'ENSAE, dans le cadre du cours "Python for Data Science". L'idée de ce projet vient d'une intuition que nous avons eu selon laquelle il était possible de prédire le succès d'un jeu vidéo auprès des utilisateurs à partir de certaines de ses caractéristiques. L'objectif principal de ce travail est donc de tenter de vérifier ou infirmer cette intuition à l'aide de techniques statistiques de traitement de données. 

De plus, nous voulions y incorporer une partie plus originale (traitement d'images...)

# Importation des modules 

## Modules de webscrapping

On installe les packages utiles au Webscrapping des données sur les jeux

In [563]:
# On importe les packages
!pip install unicode
!pip install unidecode
import requests 
import urllib
import bs4
from requests_html import HTMLSession
from tqdm import tqdm
from unidecode import unidecode
import datetime
from igdb.wrapper import IGDBWrapper
from igdb.igdbapi_pb2 import GenreResult
from igdb.igdbapi_pb2 import ThemeResult
from igdb.igdbapi_pb2 import GameResult
from igdb.igdbapi_pb2 import InvolvedCompanyResult
from igdb.igdbapi_pb2 import PlayerPerspectiveResult
from igdb.igdbapi_pb2 import MultiplayerModeResult
from igdb.igdbapi_pb2 import ArtworkResult
from igdb.igdbapi_pb2 import AgeRatingResult



## Importation des modules de Visualisation & Modélisation des données

In [498]:
#Là on importe les autres packages
import numpy as np
import pandas as pd

# I/ Récupération des données

## I/A Webscrapping des titres des jeux vidéos sur Wikipédia

A partir du site Wikipédia, nous allons récupérer les titres de tous les jeux vidéos sortis depuis l'année 2000. Par exemple, depuis l'URL : https://en.wikipedia.org/wiki/Category:2023_video_games nous avons accès à la liste de la quasi-totalité des jeux vidéos sortis en 2023. Nous allons donc modifier l'URL pour chaque année et constituer une liste (Liste_VG)

In [499]:
#Url initial : https://en.wikipedia.org/wiki/2023_in_video_games
Liste_VG = []

d = {17: 11, 18 : 9, 19 : 9, 20: 12, 21: 11, 22 : 7}
for i in range(17, 23) :
    
    
    response = requests.get(url="https://en.wikipedia.org/wiki/20"+str(i)+"_in_video_games")
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    tableau_participants = soup.findAll('table', {'class' : 'wikitable'})
    for j in range(d[i], d[i]+4) :
        rows = tableau_participants[j].find_all('tr')
        first = True
        for row in rows :
            if first :
                first = False
            else:
                title = row.find('i')
                if title != None :
                    Liste_VG.append((row.find('i').text))
                
                    
print(Liste_VG)


['Milkmaid of the Milky Way', 'Hatsune Miku: Project DIVA Future Tone', 'Criminal Girls: Invite Only', 'Danganronpa V3: Killing Harmony', 'School Girl/Zombie Hunter', 'Rise & Shine', 'Atelier Shallie Plus: Alchemists of the Dusk Sea', 'Fate/Extella: The Umbral Star', 'Mighty Morphin Power Rangers: Mega Battle', 'Birthdays the Beginning', 'Valkyria Revolution', 'Dragon Quest VIII: Journey of the Cursed King', 'Gravity Rush 2', 'Urban Empire', 'Don Bradman Cricket 17', 'Kingdom Hearts HD 2.8 Final Chapter Prologue', 'Pokémon Duel', 'Resident Evil 7: Biohazard', 'Tales of Berseria', 'Yakuza 0', 'Memoranda', 'Twin Star Exorcists', 'Disgaea 2', 'Double Dragon IV', 'Angels with Scaly Wings', 'Digimon World: Next Order', 'Divide', 'Dynasty Warriors: Godseekers', 'Poi', 'Fire Emblem Heroes', 'A House of Many Doors', 'Husk', "Poochy & Yoshi's Woolly World", 'Atelier Sophie: The Alchemist of the Mysterious Book', 'Nights of Azure', 'Nioh', 'Reconstrucción', 'Splasher', 'WWE 2K17', 'Bendy and the

In [500]:
print(tableau_participants[8].find_all('th'))

#ce que j'ai rajouté pour extraire les autres trucs de wikipédia mais pue un peu la merde
#col = row.find_all(['td', 'th'])
                #if len(col) >= 2 and title != None and col[1].name and col[2] != None :
                    #Liste_VG.append([title.text, col[1].text, col[2].text])


#print(platforms[0].text)

[<th>Month</th>, <th>Day</th>, <th>Title</th>, <th>Platform(s)</th>, <th>Genre(s)</th>, <th>Developer(s)</th>, <th>Publisher(s)</th>, <th><abbr title="Reference(s)">Ref.</abbr>
</th>]


## I/B Récupération des notes des jeux

L'objectif de cette partie est d'extraire du site "Métacritic", un site spécialisé qui répertorie les avis de professionnels et d'utilisateurs sur les nouvelles sorties jeux vidéos. Pour chaque jeu dans la base de données du site, une note sur 100 est attribuée (moyenne des reviews de sites spécialisés jeux vidéos) et une note sur 10 attribuée par les utilisateurs du site. Nous allons ici récupérer pour chaque jeu ses notes Métacritic qui deviendront les variables que nous tenteront de prédire par la suite. 

On remarque que l'on peut atteindre la review du jeu à partir de l'URL, il suffit d'indiquer le titre du jeu dans l'URL, en faisant attention aux espaces. 

Ici, nous allons remarquer que l'accès à Métacritic est interdit via la bibliothèque Beautifoul Soup

In [501]:
#URL du site : https://www.metacritic.com/game/
#URL d'une recherche : https://www.metacritic.com/search/gran%20theft%20auto/
#URL d'une review : https://www.metacritic.com/game/grand-theft-auto-v/
#URL d'une review d'un autre site spécialisé : https://www.ign.com/games/grand-theft-auto-v

#Base URL de toutes les reviews
URL = "https://www.metacritic.com/game/"

#user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
#site = requests.get(URLbis, user_agent)
#print(site.status_code)

On crée une fonction afin de reformater les titres des jeux pour qu'ils correspondent au format des URL Métacritic

In [502]:
def title_to_slug(title):
    
    title = title.lower()
    title = title.replace(':', '')
    title = title.replace('•', '-')
    title = title.replace(' – ', '')
    #ce ne sont pas exactement les mêmes tirets ici
    title = title.replace(' — ', '')
    title = title.replace(' - ', '')
    title = title.replace('/', '-')
    title = title.replace('.', '')
    title = title.replace('&', 'and')
    title = title.replace("'", '')
    title = title.replace('+', 'plus')
    title = title.replace('ō', 'o')
    title = title.replace(' ', '-')
    
    return(str(title))


On définit cette fonction qui va, pour une liste de titres de jeux donnée, récupérer les notes Métacritic sur la page web de la review du jeu en question. Ce sont ces notes que nous allons essayer de prédire par la suite car elles correspondent globablement à la mesure de l'accueil et de la qualité du jeu par les joueurs. 

Afin d'accélérer le code et sachant que nous allons utiliser uniquement le métascore qui reflète les avis de professionnels du milieu du jeu vidéo, nous allons ajouter à l'utilisateur la possibilité de choisir d'extraire le métascore ou le userscore

In [503]:
print('—' == '–')

False


In [504]:
print('-' == '–')

False


In [505]:
def recup_metascore(list, param) : 
    session = HTMLSession()
    liste_ratings = []

    if param == 1 :

        for i in range(len(list)) :
            #on reformate les titres extraits pour qu'ils aient le format des URL Métacritic
            title = unidecode(list[i])
            Url = URL + title_to_slug(title) + '/'

            #on get l'url
            DOM_Html = session.get(Url)
            chemin_metascore = DOM_Html.html.find('#__layout > div > div.c-layoutDefault_page > div.c-pageProductGame > div:nth-child(1) > div > div \
                                    > div.c-productHero_player-scoreInfo.u-grid.g-grid-container > \
                                    div.c-productHero_score-container.u-flexbox.u-flexbox-column.g-bg-white > \
                                    div.c-productHero_scoreInfo.g-inner-spacing-top-medium.g-outer-spacing-bottom-medium.g-outer-spacing-top-medium \
                                    > div:nth-child(1) > div > div.c-productScoreInfo_scoreContent.u-flexbox.u-flexbox-alignCenter.u-flexbox-justifyFlexEnd.g-width-100.u-flexbox-nowrap > \
                                    div.c-productScoreInfo_scoreNumber.u-float-right > div > div', first = True)
            #on extrait le rating
            if chemin_metascore != None :
                metacritic_rating_inter = chemin_metascore.attrs['title']
                metacritic_rating = metacritic_rating_inter.split(' ')[1]
            else : 
                metacritic_rating = 'N/A'

            liste_ratings.append([list[i], metacritic_rating])

    elif param == 2 : 

        for i in range(len(list)) :
            #on reformate les titres extraits pour qu'ils aient le format des URL Métacritic
            title = unidecode(list[i])
            Url = URL + title_to_slug(title) + '/'

            #on get l'url
            DOM_Html = session.get(Url)
            chemin_userscore = DOM_Html.html.find('#__layout > div > div.c-layoutDefault_page > div.c-pageProductGame > div:nth-child(1) > div > div > \
                                      div.c-productHero_player-scoreInfo.u-grid.g-grid-container > div.c-productHero_score-container.u-flexbox.u-flexbox-column.g-bg-white > \
                                      div.c-productHero_scoreInfo.g-inner-spacing-top-medium.g-outer-spacing-bottom-medium.g-outer-spacing-top-medium > \
                                      div.c-productScoreInfo.u-clearfix > div.c-productScoreInfo_scoreContent.u-flexbox.u-flexbox-alignCenter.u-flexbox-justifyFlexEnd.g-width-100.u-flexbox-nowrap > \
                                      div.c-productScoreInfo_scoreNumber.u-float-right > div > div' , first = True)
            #On extrait le rating
            if chemin_userscore != None :  
                users_ratings_inter = chemin_userscore.attrs['title']
                users_ratings = float(users_ratings_inter.split(' ')[2])
            else:
                users_ratings = 'N/A'
        
            liste_ratings.append([list[i],  users_ratings])


    return liste_ratings

    

Pas trop capté ici

In [506]:
#from igdb.wrapper import IGDBWrapper
wrapper = IGDBWrapper("wlqlv1d19z5t69oqlf9xx69znfahze", "o2strnp2xj90l3p982n4jg9ko1utad")

Création du dictionnaire des différents genres de jeux : chaque genre est associé à un Id qui nous permettra de nous repérer dans le base

In [507]:
byte_array = wrapper.api_request(
            'genres.pb',
            'fields *; limit 100;'
          )
genres_message = GenreResult()
genres_message.ParseFromString(byte_array) # Fills the protobuf message object with the response
genres = genres_message.genres
genre_nbr = len(genres)
genre_dictionnary = {}

for i in range(genre_nbr) :
    genre_dictionnary[genres[i].id] = genres[i].name
print(genre_dictionnary)

{4: 'Fighting', 5: 'Shooter', 7: 'Music', 8: 'Platform', 9: 'Puzzle', 10: 'Racing', 11: 'Real Time Strategy (RTS)', 12: 'Role-playing (RPG)', 13: 'Simulator', 14: 'Sport', 15: 'Strategy', 16: 'Turn-based strategy (TBS)', 24: 'Tactical', 26: 'Quiz/Trivia', 25: "Hack and slash/Beat 'em up", 30: 'Pinball', 31: 'Adventure', 33: 'Arcade', 34: 'Visual Novel', 32: 'Indie', 35: 'Card & Board Game', 36: 'MOBA', 2: 'Point-and-click'}


Même chose pour le thème principal du jeu

In [508]:
byte_array = wrapper.api_request(
            'themes.pb',
            'fields *; limit 100;'
          )
themes_message = ThemeResult()
themes_message.ParseFromString(byte_array)# Fills the protobuf message object with the response
themes = themes_message.themes
theme_nbr = len(themes)
theme_dictionnary = {}

for i in range(theme_nbr) :
    theme_dictionnary[themes[i].id] = themes[i].name
print(theme_dictionnary)

{20: 'Thriller', 18: 'Science fiction', 1: 'Action', 19: 'Horror', 21: 'Survival', 17: 'Fantasy', 22: 'Historical', 23: 'Stealth', 27: 'Comedy', 28: 'Business', 31: 'Drama', 32: 'Non-fiction', 35: 'Kids', 33: 'Sandbox', 38: 'Open world', 39: 'Warfare', 41: '4X (explore, expand, exploit, and exterminate)', 34: 'Educational', 43: 'Mystery', 40: 'Party', 44: 'Romance', 42: 'Erotic'}


Fonction aller chercher nb_pixels dans la table adéquat

In [516]:
def fetch_artwork(id) : 
    byte_array = wrapper.api_request(
            'artworks.pb',
            f'fields *; where id = {id};'
          )
    artwork_message = ArtworkResult()
    artwork_message.ParseFromString(byte_array)# Fills the protobuf message object with the response
    artworks = artwork_message.artworks
    longueur = artworks[0].width
    largeur = artworks[0].height
    nb_pixels = largeur * longueur

    return nb_pixels
    

Fonction pour récupérer le age_rating dans la base

In [604]:
dictionnary_rating = {}
dictionnary_rating[4] = '16 year'
dictionnary_rating[21] = '16 year'
dictionnary_rating[25] = '16 year'
dictionnary_rating[32] = '16 year'
dictionnary_rating[5] = '18 year'
dictionnary_rating[26] = '18 year'
dictionnary_rating[33] = '18 year'
dictionnary_rating[22] = '18 year'

In [605]:
def fetch_age_ratings(id) :
    byte_array = wrapper.api_request(
                'age_ratings.pb',
                f'fields *; where id = {id};'
              )
    age_rating_message = AgeRatingResult()
    age_rating_message.ParseFromString(byte_array)# Fills the protobuf message object with the response
    ageratings = age_rating_message.ageratings

    if ageratings[0].rating in dictionnary_rating :
        PEGI = dictionnary_rating[ageratings[0].rating]
    else : 
        PEGI = 'Tout Public'

    return(PEGI)

On trouve l'info sur le site de l'API de la signification des id_rating

Fonction pour récupérer player_perspective dans la base 

In [643]:
def fetch_player_perspective(id) :
    byte_array = wrapper.api_request(
                'player_perspectives.pb',
                f'fields *; where id = {id};'
              )
    pp_message = PlayerPerspectiveResult()
    pp_message.ParseFromString(byte_array)# Fills the protobuf message object with the response
    playerperspectives = pp_message.playerperspectives

    pp = playerperspectives[0].name

    return(pp)

Ici, il s'agit de la requête que l'on fait à l'API qui va extraire, à partir du nom du jeu, les informations correspondantes dans sa base de données. 

In [644]:
fetch_player_perspective(7)

'Virtual Reality'

In [653]:
import datetime

def game_API_Info(game_slug) :

    #Protobuf API request
    from igdb.igdbapi_pb2 import GameResult
    #On fait cela non pas pour préciser si le jeu n'ai pas dispo sur l'api mais si jamais des caractères dans son titre empêchent sa lecture 
    #(utile pour affiner la fonction de reformatage des titres)
    try : 
        byte_array = wrapper.api_request(
                'games.pb',
                'fields first_release_date, franchise, genres, hypes, language_supports, platforms, themes, age_ratings, collection, artworks, \
                involved_companies, multiplayer_modes, player_perspectives, similar_games, summary, storyline \
                ; where slug = "'+ game_slug +'";'
              )
    except: 
        print(f"Une erreur d'extraction s'est produite pour le jeu {game_slug}")
        return(["NA"])
    games_message = GameResult()
    games_message.ParseFromString(byte_array) # Fills the protobuf message object with the response
    games = games_message.games
    
    if len(games) == 0 :
        return(["NA"])
    unix_time = games[0].first_release_date
    seconds = unix_time.seconds
    dt_object = datetime.datetime.utcfromtimestamp(seconds)
    year = dt_object.year
    
    genres_nbr = len(games[0].genres)
    genres_names_list = []
    for j in range(genres_nbr):
        genres_names_list.append(genre_dictionnary[games[0].genres[j].id])

    themes_nbr = len(games[0].themes)
    themes_names_list = []
    for j in range(themes_nbr):
        themes_names_list.append(theme_dictionnary[games[0].themes[j].id])
        
    #requête pour le nb de pixels
    if games[0].artworks == [] : 
        nb_pixels = 'NA'
    else : 
        nb_pixels = fetch_artwork(games[0].artworks[0].id)

    #requête pegi rating
    if games[0].age_ratings == [] : 
        age_rating_pegi = 'NA'
    else : 
        for i in range(len(games[0].age_ratings)): 
            if fetch_age_ratings(games[0].age_ratings[i].id) == 'Tout Public': 
                age_rating_pegi_bis = 'Tout Public'
            else : 
                age_rating_pegi_bis
        age_rating_pegi = age_rating_pegi_bis

    #requête player_perspective
    if games[0].player_perspectives == [] : 
        player_perspectives = 'NA'
    else : 
        player_perspectives = fetch_player_perspective(games[0].player_perspectives[0].id)
    
    return([games[0].id, year, age_rating_pegi, games[0].franchise, genres_names_list, games[0].hypes, len(games[0].language_supports), \
            len(games[0].platforms), themes_names_list, games[0].collection, nb_pixels, \
            games[0].involved_companies, games[0].multiplayer_modes, player_perspectives, \
            games[0].similar_games, games[0].summary, games[0].storyline])

Ici on va créer le dataframe qui rassemble les informations disponibles sur la base de l'API ainsi que les notes Métacritic des jeux

In [646]:
def creation_dataframe(list) : 
    np_df = []
    k = 0

    for i in tqdm(range(len(list))) :
        data_recup = recup_metascore([Liste_VG[i]], 1)
        titre = data_recup[0][0]
        metascore = data_recup[0][1]
        slug = title_to_slug(Liste_VG[i])
        vect_info_jeu = game_API_Info(slug)
        #Ici on vient filtrer les jeux qui ne sont pas répertoriés dans la base de données (vecteur d'informations = 'NA')
        
        if len([titre] + [metascore] + vect_info_jeu) == 19 : 
            np_df.append([titre] + [metascore] + game_API_Info(slug))
        elif vect_info_jeu == ['NA'] : 
            k += 1
        
    np_DF = pd.DataFrame(np.array(np_df), columns = ['Title', 'Note', 'Id', 'Release_Date', 'PEGI', 'Franchise', 'Genres', 'Hypes', 'nb_languages', \
                                                    'nb_platforms', 'Themes', 'Collection', 'Graphismes', 'Publisher', 'Multiplayer', \
                                                    'Perspective', 'Similar Games', 'Summary', 'Storyline'])
    
    print(f"La proportion de jeux qui ont matché avec la base de l'API est de {int(1000*(len(list) - k)/len(list))/10} %")
    return(np_DF)

In [654]:
game_API_Info(title_to_slug(Liste_VG[2]))

[11746,
 2013,
 'Tout Public',
 ,
 ['Role-playing (RPG)', 'Simulator', 'Strategy'],
 0,
 5,
 2,
 ['Action', 'Erotic'],
 id: 1573,
 1190400,
 [id: 25492
 , id: 25493
 , id: 43867
 ],
 [],
 'Bird view / Isometric',
 [id: 22472
 , id: 25311
 , id: 28465
 , id: 35994
 , id: 41895
 , id: 43097
 , id: 43130
 , id: 51991
 , id: 54815
 , id: 67166
 ],
 'Criminal Girls: Invite Only a Japanese role-playing game and the enhanced version of the original Criminal Girls game. The game follows a group of girls in Hell who have to climb a tower to be reincarnated. The battle system is unique since you give the characters, each representing a different vice, orders in battle and sometimes they won’t listen to you because they haven’t been trained.',
 '']

In [609]:
creation_dataframe(Liste_VG[:30])

100%|██████████| 30/30 [01:59<00:00,  4.00s/it]


['Fire Emblem Heroes', '72', 26841, 2017, 'Tout Public', , ['Role-playing (RPG)', 'Simulator', 'Strategy', 'Turn-based strategy (TBS)'], 1, 0, 3, ['Action', 'Fantasy', 'Historical', 'Warfare'], , 1608000, [id: 43772
, id: 43773
, id: 46490
], [], [id: 3
], [id: 8746
, id: 9243
, id: 10388
, id: 14394
, id: 18623
, id: 23345
, id: 26845
, id: 26940
, id: 27209
, id: 36553
], '"Take part in strategic turn-based battles streamlined for on-the-go play with maps that fit in the palm of your hand! Lead your army with easy touch and drag controls, including the ability to attack by simply swiping an ally over an enemy."', '"A world with two kingdoms: the Emblian Empire, which wishes to rule all worlds, and the Askran Kingdom, which stands in its way.\nYou are a summoner with the special ability to call upon legendary Heroes from different Fire Emblem worlds. In order to save the Kingdom of Askr from destruction, join the Order of Heroes and face a never-ending challenge."']
La proportion de j

/var/folders/sg/lxw89kss0x727nlhml9vq0nm0000gn/T/ipykernel_65572/716728829.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np_DF = pd.DataFrame(np.array(np_df), columns = ['Title', 'Note', 'Id', 'Release_Date', 'PEGI', 'Franchise', 'Genres', 'Hypes', 'nb_languages', \


,Title,Note,Id,Release_Date,PEGI,Franchise,Genres,Hypes,nb_languages,nb_platforms,Themes,Collection,Graphismes,Publisher,Multiplayer,Perspective,Similar Games,Summary,Storyline
0,Milkmaid of the Milky Way,74,26732,2017,Tout Public,,"[Point-and-click, Puzzle, Adventure, Indie]",0,6,6,"[Fantasy, Science fiction]",,2073600,[id: 96211\n],[],[id: 4\n],"[id: 236\n, id: 11367\n, id: 18011\n, id: 2442...",Milkmaid of the Milky Way is a classic puzzle ...,
1,Hatsune Miku: Project DIVA Future Tone,N/A,25821,2016,Tout Public,,[Music],0,5,1,[Action],id: 1292\n,2073600,"[id: 222786\n, id: 239228\n]",[],[id: 2\n],"[id: 4052\n, id: 11711\n, id: 18981\n, id: 280...",Hatsune Miku: Project Diva Future Tone is a ho...,"August 31, 2007 marks the 10th anniversary of ..."
2,Criminal Girls: Invite Only,55,11746,2013,Tout Public,,"[Role-playing (RPG), Simulator, Strategy]",0,5,2,"[Action, Erotic]",id: 1573\n,1190400,"[id: 25492\n, id: 25493\n, id: 43867\n]",[],[id: 3\n],"[id: 22472\n, id: 25311\n, id: 28465\n, id: 35...",Criminal Girls: Invite Only a Japanese role-pl...,
3,Rise & Shine,66,6732,2017,Tout Public,,"[Shooter, Platform, Puzzle, Adventure, Indie, ...",2,12,6,"[Action, Fantasy]",,2073600,"[id: 128885\n, id: 128887\n]",[],[id: 4\n],"[id: 7350\n, id: 11646\n, id: 16992\n, id: 244...",The game mixes pure shooting arcade with the u...,"Rise is a small kid of Gamearth, the world whe..."
4,Atelier Shallie Plus: Alchemists of the Dusk Sea,78,25699,2016,Tout Public,,"[Role-playing (RPG), Adventure]",0,0,1,[Fantasy],id: 2501\n,1143000,"[id: 41425\n, id: 41426\n, id: 41427\n]",[],[id: 2\n],"[id: 7276\n, id: 24426\n, id: 25921\n, id: 280...","""Atelier Shallie Plus -Alchemists of the Dusk ...",
5,Birthdays the Beginning,65,26611,2017,Tout Public,,"[Simulator, Strategy]",0,18,3,"[Sandbox, Open world]",,1190400,"[id: 43173\n, id: 43174\n, id: 43175\n]",[],[id: 3\n],"[id: 10032\n, id: 26574\n, id: 34823\n, id: 37...","""From the legendary creator, Mr. Yasuhiro Wada...",
6,Valkyria Revolution,54,14673,2017,Tout Public,,"[Role-playing (RPG), Strategy]",2,0,3,"[Action, Fantasy, Warfare]",id: 1877\n,2073600,"[id: 184678\n, id: 184679\n, id: 184680\n, id:...",[],[id: 2\n],"[id: 19164\n, id: 19404\n, id: 24426\n, id: 25...","A new spin off game in the Valkyria franchise,...",All it takes is a single spark to start the in...
7,Dragon Quest VIII: Journey of the Cursed King,89,1819,2004,Tout Public,,"[Role-playing (RPG), Strategy, Adventure]",0,0,1,"[Action, Fantasy]",,560000,"[id: 45936\n, id: 175892\n]",[],[id: 2\n],"[id: 421\n, id: 425\n, id: 426\n, id: 832\n, i...",Dragon Quest VIII: Journey of the Cursed King ...,The court jester Dhoulmagus of the kingdom of ...
8,Gravity Rush 2,80,12528,2017,Tout Public,,"[Platform, Adventure]",18,0,1,[Action],id: 1664\n,1121400,"[id: 39484\n, id: 39485\n]",[],[id: 2\n],"[id: 18869\n, id: 20342\n, id: 24426\n, id: 28...",Another mind-bending adventure awaits gravity ...,
9,Urban Empire,62,18425,2017,NA,,"[Simulator, Strategy]",5,15,1,[],,1190400,"[id: 37935\n, id: 58727\n]",[id: 7573\n],[id: 3\n],"[id: 17519\n, id: 17613\n, id: 19376\n, id: 29...",In Urban Empire you take control of a mayoral ...,"Beginning in the 1820s, establish your city th..."
